### TITLE HERE

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import metrics
from scipy.stats import ks_2samp
from scipy.stats import t
import matplotlib.ticker as mticker
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_curve, accuracy_score
from sklearn.impute import KNNImputer
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV, StratifiedKFold
plt.style.use('ggplot')default, 1: Defaulted on a loan or payment</td></tr>
</table>


In [ ]:
# S3 bucket and object key
bucket_name = 'dataset-content-pedrohang'
object_key = 'credit-scoring/credit_scoring.ftr'

# Constructing the S3 URI
s3_uri = f's3://{bucket_name}/{object_key}'

# Reading the feather file from S3
df = pd.read_feather(s3_uri, storage_options={'anon': True})

df.drop(columns=['index'], inplace=True)

df['default'] = df.default.astype('int64')

df['not_default'] = 1 - df['default']

def drop_missing_values(my_df):
    # Drop rows with any missing values
    my_df.dropna(inplace=True)
    
    return my_df

df_filled = drop_missing_values(df)

df_train = df_filled[df_filled['reference_date'] <= '2015-12-31']
df_oot = df_filled[df_filled['reference_date'] >= '2016-01-01']misleading the model's performance and interpretation.on

def IV(x, y):
    # Create a crosstab to calculate the frequency distribution of x and y
    table = pd.crosstab(x, y, margins=True, margins_name='total')

    # Get the labels for the event and non-event categories
    event_label = table.columns[0]
    non_event_label = table.columns[1]

    # Calculate the percentage of events
    table['pct_event'] = table[event_label] / table.loc['total', event_label]
    table['event_proportion'] = table[event_label] / table.loc['total', event_label]

    # Calculate the percentage of non-events
    table['pct_non_event'] = table[non_event_label] / table.loc['total', non_event_label]

    # Calculate Weight of Evidence (WoE)
    table['woe'] = np.log(table.pct_event / table.pct_non_event)

    # Calculate the partial Information Value (IV)
    table['iv_partial'] = (table.pct_event - table.pct_non_event) * table.woe

    # Return the sum of the partial IVs as the total Information Value
    return table['iv_partial'].sum()

metadata = pd.DataFrame(df_filled.dtypes, columns=['dtype'])
metadata['nmissing'] = df_filled.isna().sum()
metadata['unique_values'] = df_filled.nunique()
metadata['role'] = 'covariate'
metadata.loc['default','role'] = 'response'
metadata.loc['not_default','role'] = 'response'
metadata

for var in metadata[metadata.role=='covariate'].index:
    if  (metadata.loc[var, 'unique_values']>6):
        metadata.loc[var, 'IV'] = IV(pd.qcut(df_filled[var],5,duplicates='drop'), df_filled.default)
    else: 
        metadata.loc[var, 'IV'] = IV(df_filled[var], df_filled.default)

    
metadata

def biv_discrete(var, df_filled):
    df_filled['not_default'] = 1 - df_filled.default
    g = df_filled.groupby(var)

    biv = pd.DataFrame({'qt_not_default': g['not_default'].sum(),
                        'qt_default': g['default'].sum(),
                        'default': g['default'].mean(),
                        var: g['default'].mean().index, 
                        'cont': g[var].count()})
    
    biv['ep'] = (biv.default * (1 - biv.default) / biv.cont) ** .5
    biv['default_sup'] = biv.default + t.ppf([0.975], biv.cont - 1) * biv.ep
    biv['default_inf'] = biv.default - t.ppf([0.025], biv.cont - 1) * biv.ep
    
    biv['logit'] = np.log(biv.default / (1 - biv.default))
    biv['logit_sup'] = np.log(biv.default_sup / (1 - biv.default_sup))
    biv['logit_inf'] = np.log(biv.default_inf / (1 - biv.default_inf))

    general_default_rate = df_filled.default.mean()
    general_woe = np.log(df_filled.default.mean() / (1 - df_filled.default.mean()))

    biv['woe'] = biv.logit - general_woe
    biv['woe_sup'] = biv.logit_sup - general_woe
    biv['woe_inf'] = biv.logit_inf - general_woe

    fig, ax = plt.subplots(2, 1, figsize=(8, 6))
    ax[0].plot(biv[var], biv.woe, ':bo', label='woe')
    ax[0].plot(biv[var], biv.woe_sup, 'o:r', label='superior limit')
    ax[0].plot(biv[var], biv.woe_inf, 'o:r', label='inferior limit')
    
    num_cat = biv.shape[0]
    ax[0].set_xlim([-.3, num_cat - .7])

    ax[0].set_ylabel("Weight of Evidence")
    ax[0].legend(bbox_to_anchor=(.83, 1.17), ncol=3)
    
    ax[0].set_xticks(list(range(num_cat)))
    ax[0].set_xticklabels(biv[var], rotation=15)
    
    ax[1] = biv.cont.plot.bar(ax=ax[1])

    # Adding count labels on top of each bar
    for p in ax[1].patches:
        ax[1].annotate(f'{int(p.get_height())}', 
                       (p.get_x() + p.get_width() / 2., p.get_height()), 
                       ha='center', va='center', 
                       xytext=(0, 10), 
                       textcoords='offset points')
    
    return biv


df_struc = df_filled.copy()

biv_discrete('income_type', df_struc)

df_struc.income_type.replace({'Scholarship Holder': 'Scholarship/P.Servant', 'Public Servant': 'Scholarship/P.Servant'}, inplace=True)
biv_discrete('income_type', df_struc)

biv_discrete('residents_count', df_struc)

df_struc['residents_count'] = df_struc['residents_count'].astype(float)
df_struc['residents_count'] = df_struc['residents_count'].apply(lambda x: x if x <= 6 else '>6')
df_struc['residents_count'] = df_struc['residents_count'].astype(str)
biv_discrete('residents_count', df_struc)

def biv_continuous(var, ncat, df):
    df['not_default'] = 1 - df.default
    cat_srs, bins = pd.qcut(df[var], ncat, retbins=True, precision=0, duplicates='drop')
    
    # Explicitly set observed=False to silence the warning
    g = df.groupby(cat_srs, observed=False)

    biv = pd.DataFrame({'qt_not_default': g['not_default'].sum(),
                        'qt_default': g['default'].sum(),
                        'default': g['default'].mean(), 
                        var: g[var].mean(), 
                        'cont': g[var].count()})
    
    biv['ep'] = (biv.default * (1 - biv.default) / biv.cont) ** .5
    biv['default_sup'] = biv.default + t.ppf([0.975], biv.cont - 1) * biv.ep
    biv['default_inf'] = biv.default - t.ppf([0.025], biv.cont - 1) * biv.ep
    
    biv['logit'] = np.log(biv.default / (1 - biv.default))
    biv['logit_sup'] = np.log(biv.default_sup / (1 - biv.default_sup))
    biv['logit_inf'] = np.log(biv.default_inf / (1 - biv.default_inf))

    general_default_rate = df.default.mean()
    general_woe = np.log(df.default.mean() / (1 - df.default.mean()))

    biv['woe'] = biv.logit - general_woe
    biv['woe_sup'] = biv.logit_sup - general_woe
    biv['woe_inf'] = biv.logit_inf - general_woe

    fig, ax = plt.subplots(2, 1, figsize=(8, 6))
    ax[0].plot(biv[var], biv.woe, ':bo', label='woe')
    ax[0].plot(biv[var], biv.woe_sup, 'o:r', label='superior limit')
    ax[0].plot(biv[var], biv.woe_inf, 'o:r', label='inferior limit')
    
    num_cat = biv.shape[0]

    ax[0].set_ylabel("Weight of Evidence")
    ax[0].legend(bbox_to_anchor=(.83, 1.17), ncol=3)
    
    ax[1] = biv.cont.plot.bar()
    
    return None


biv_continuous('employment_time', 15, df_struc)

df_struc.loc[df_struc['employment_time']<0,'employment_time'] = -1

df_struc = df_struc.drop(columns=['income'])

def remove_outliers_zscore(df, z_thresh=3.0):
    df_cleaned = df.copy()

    removal_counts = {}

    for column in df_cleaned.select_dtypes(include=[np.number]).columns:
        z_scores = np.abs(stats.zscore(df_cleaned[column]))

        outliers = z_scores > z_thresh
        num_outliers = outliers.sum()
        
        if num_outliers > 0:
            removal_counts[column] = num_outliers

        df_cleaned = df_cleaned[~outliers]

    for column, count in removal_counts.items():
        print(f"Removed {count} rows based on outliers in column '{column}'")

    df_cleaned.reset_index(drop=True, inplace=True)
    
    return df_cleaned

df_outl = remove_outliers_zscore(df_struc, 4.0)

---

def one_hot_encode(df):
    df_encoded = pd.get_dummies(df, drop_first=True)

    return df_encoded

df_1hot = one_hot_encode(df_outl)

df_1hot.columns

len(df_1hot.columns)

continuous_columns = ['age', 'income', 'employment_time', 'children_count']

dummy_columns = [col for col in df_1hot.columns if col not in continuous_columns + ['not_default', 'default']]

scaler = StandardScaler()
df_scaled_continuous = pd.DataFrame(scaler.fit_transform(df_1hot[continuous_columns]), columns=continuous_columns)

df_combined = pd.concat([df_scaled_continuous, df_1hot[dummy_columns].reset_index(drop=True), df_1hot[['not_default', 'default']].reset_index(drop=True)], axis=1)

df_train = df_combined[df_combined['reference_date'] <= '2015-12-31']
df_oot = df_combined[df_combined['reference_date'] >= '2016-01-01']

df_train = df_train.drop(columns=['not_default', 'reference_date', 'income'], inplace=False)
df_oot = df_oot.drop(columns=['not_default', 'reference_date', 'income'], inplace=False)

X = df_train.drop(columns=['default'])
y = df_train['default']  # This is the response variable

pca = PCA(n_components=10)
X_pca = pca.fit_transform(X)

df_pca = pd.DataFrame(data=X_pca, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10'])

df_train_final = pd.concat([df_pca, y.reset_index(drop=True)], axis=1)

X_ = df_oot.drop(columns=['default'])
y_ = df_oot['default']  # This is the response variable
pca_ = PCA(n_components=10)
X_pca_ = pca_.fit_transform(X_)
df_pca_ = pd.DataFrame(data=X_pca_, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10'])
df_oot_final = pd.concat([df_pca_, y_.reset_index(drop=True)], axis=1)

cumulative_explained_variance = np.cumsum(pca.explained_variance_ratio_)

df_train_final_X = df_train_final.drop(columns=['default'])
df_train_final_y = df_train_final['default']

df_oot_final_X = df_oot_final.drop(columns=['default'])
df_oot_final_y = df_oot_final['default']

model = LogisticRegression(class_weight='balanced')
model.fit(df_train_final_X, df_train_final_y)

predictions_train = model.predict(df_train_final_X)

# Convert probabilities to binary outcomes (threshold of 0.5)
predicted_classes_train = (predictions_train >= 0.5).astype(int)
accuracy_train = accuracy_score(df_train_final_y, predicted_classes_train)
print(f'Accuracy for the Train set: {accuracy_train * 100:.2f}%')

predictions_oot = model.predict(df_oot_final_X)

# Convert probabilities to binary outcomes (threshold of 0.5)
predicted_classes_oot = (predictions_oot >= 0.5).astype(int)
accuracy_oot = accuracy_score(df_oot_final_y, predicted_classes_oot)
print(f'Accuracy for the Out-of-Time set: {accuracy_oot * 100:.2f}%')

# Confusion matrix for the training set
cm_train = confusion_matrix(df_train_final_y, predictions_train >= 0.5)
# Confusion matrix for the OOT set
cm_oot = confusion_matrix(df_oot_final_y, predictions_oot >= 0.5)

fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# Training set confusion matrix
ConfusionMatrixDisplay(cm_train, display_labels=['No Default', 'Default']).plot(ax=ax[0], cmap='Blues')
ax[0].set_title('Training Set Confusion Matrix')

# OOT set confusion matrix
ConfusionMatrixDisplay(cm_oot, display_labels=['No Default', 'Default']).plot(ax=ax[1], cmap='Greens')
ax[1].set_title('OOT Set Confusion Matrix')

plt.show()


## KS

The <b>Kolmogorov-Smirnov (KS)</b> statistic is a metric used to evaluate the discriminatory power of a logistic regression model. It measures the maximum distance between the cumulative distribution functions (CDFs) of the predicted probabilities for the positive and negative classes. A higher KS value indicates better separation between the classes, reflecting the model's effectiveness in distinguishing between them.

#### KS Statistic Interpretation

The divisions below provide a general guideline for interpreting KS values in the context of model performance:

- **KS < 0.1:** Poor separation; the model has limited ability to distinguish between classes.
- **0.1 ≤ KS < 0.2:** Fair separation; the model shows some ability to distinguish between classes but may need improvement.
- **0.2 ≤ KS < 0.3:** Good separation; the model effectively discriminates between classes.
- **KS ≥ 0.3:** Excellent separation; the model is highly effective at distinguishing between classes.

These divisions are commonly used in industries such as finance and risk modeling to assess the effectiveness of predictive models.


prob_train_pos = predictions_train[df_train_final_y == 1]  # Predicted probabilities for positive class
prob_train_neg = predictions_train[df_train_final_y == 0]  # Predicted probabilities for negative class
ks_stat_train = stats.ks_2samp(prob_train_pos, prob_train_neg).statistic
print(f'KS Statistic for Training Set: {ks_stat_train:.4f}')

prob_train_pos = predictions_oot[df_oot_final_y == 1]  # Predicted probabilities for positive class
prob_train_neg = predictions_oot[df_oot_final_y == 0]  # Predicted probabilities for negative class
ks_stat_train = stats.ks_2samp(prob_train_pos, prob_train_neg).statistic
print(f'KS Statistic for Out-of-Time Set: {ks_stat_train:.4f}')

##### Our KS sits on the "Good Separation" category

## Gini

The **Gini coefficient** is a key metric used to evaluate the discriminatory power of a logistic regression model, showing how effectively the model distinguishes between positive and negative classes. It is closely related to the **ROC Curve** (Receiver Operating Characteristic Curve) and is calculated using the formula:

$$
\text{Gini} = 2 \times \text{AUC} - 1
$$

In an ideal scenario, where the model perfectly separates the classes, the ROC curve would touch the top left corner of the plot, yielding an **AUC** of 1 and a **Gini coefficient** of 1, signifying perfect discrimination.


### Gini Coefficient Divisions

- **Gini < 0.1:** **Very Poor** discrimination; the model has little to no ability to distinguish between classes.
- **0.1 ≤ Gini < 0.2:** **Poor** discrimination; the model shows some ability to distinguish between classes, but it may not be very effective.
- **0.2 ≤ Gini < 0.3:** **Fair** discrimination; the model can moderately distinguish between classes but still has room for improvement.
- **0.3 ≤ Gini < 0.4:** **Good** discrimination; the model is reasonably effective at distinguishing between classes.
- **Gini ≥ 0.4:** **Very Good** to **Excellent** discrimination; the model is highly effective at distinguishing between classes.


Let's calculate the Gini and plot the ROC curve.

auc_train = roc_auc_score(df_train_final_y, predictions_train)
auc_oot = roc_auc_score(df_oot_final_y, predictions_oot)
gini_train = 2 * auc_train - 1
gini_oot = 2 * auc_oot - 1
print(f'Gini Coefficient for Training Set: {gini_train:.4f}')
print(f'Gini Coefficient for OOT Set: {gini_oot:.4f}')

Our model sits in between <b>Fair</b> and <b>Good</b> discrimination.

# ROC Curve for Training Set
fpr_train, tpr_train, _ = roc_curve(df_train_final_y, predictions_train)
roc_auc_train = auc(fpr_train, tpr_train)

# ROC Curve for OOT Set
fpr_oot, tpr_oot, _ = roc_curve(df_oot_final_y, predictions_oot)
roc_auc_oot = auc(fpr_oot, tpr_oot)

# Plotting the ROC Curve
plt.figure(figsize=(10, 6))

# Training Set
plt.plot(fpr_train, tpr_train, color='blue', lw=2, label=f'Train ROC curve (area = {roc_auc_train:.2f})')

# OOT Set
plt.plot(fpr_oot, tpr_oot, color='green', lw=2, label=f'OOT ROC curve (area = {roc_auc_oot:.2f})')

# Diagonal line
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--', label='Random Guessing')

# Axis Labels and Title
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid()
plt.show()


---

<div style="background-color: #15616d; color: white; padding: 20px; border-radius: 10px;">
    <h1 style="text-align: center; font-family: 'Arial', sans-serif;">🛠️ Pipeline</h1>
    <p style="font-size: 1.2em; text-align: center;">
        Here, we'll refactor part of our code into a scikit-learn Pipeline for improved reusability and streamlined processing.
    </p>
</div>

---

## Criar um pipeline utilizando o sklearn pipeline 

## Pré processamento

### Substituição de nulos (nans)

Existe nulos na base? é dado numérico ou categórico? qual o valor de substituição? média? valor mais frequente? etc





### Remoção de outliers

Como identificar outlier? Substituir o outlier por algum valor? Remover a linha?







### Seleção de variáveis

Qual tipo de técnica? Boruta? Feature importance? 







### Redução de dimensionalidade (PCA)

Aplicar PCA para reduzir a dimensionalidade para 5





### Criação de dummies

Aplicar o get_dummies() ou onehotencoder() para transformar colunas catégoricas do dataframe em colunas de 0 e 1. 
- sexo
- posse_de_veiculo
- posse_de_imovel
- tipo_renda
- educacao
- estado_civil
- tipo_residencia

### Pipeline 

Crie um pipeline contendo essas funções.

preprocessamento()
- substituicao de nulos
- remoção outliers
- PCA
- Criação de dummy de pelo menos 1 variável (posse_de_veiculo)

### Treinar um modelo de regressão logistica com o resultado

### Salvar o pickle file do modelo treinado

In [661]:
import pickle

nome_arquivo = 'model_final.pkl'
pickle.dump(model, open(nome_arquivo, 'wb'))

# Pycaret na base de dados 

Utilize o pycaret para pre processar os dados e rodar o modelo **lightgbm**. Faça todos os passos a passos da aula e gere os gráficos finais. E o pipeline de toda a transformação.



In [670]:
import pandas as pd

df = pd.read_feather('credit_scoring.ftr')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'credit_scoring.ftr'

In [ ]:
from pycaret.classification import *
models()

In [ ]:
xxx = create_model('xxx')

### Salvar o arquivo do modelo treinado

# Projeto Final

1. Subir no GITHUB todos os jupyter notebooks/códigos que você desenvolveu nesse ultimo módulo
1. Gerar um arquivo python (.py) com todas as funções necessárias para rodar no streamlit a escoragem do arquivo de treino
    - Criar um .py
    - Criar um carregador de csv no streamlit 
    - Subir um csv no streamlit 
    - Criar um pipeline de pré processamento dos dados
    - Utilizar o modelo treinado para escorar a base 
        - nome_arquivo = 'model_final.pkl'
1. Gravar um vídeo da tela do streamlit em funcionamento (usando o próprio streamlit (temos aula disso) ou qlqr outra forma de gravação).
1. Subir no Github o vídeo de funcionamento da ferramenta como README.md.
1. Subir no Github os códigos desenvolvidos. 
1. Enviar links do github para o tutor corrigir.